In [1]:
from time import time
import numpy as np
import pyvips as pv
#import memray
import histomicstk as htk
from histomicstk.saliency.tissue_detection import get_tissue_mask
import matplotlib.pyplot as plt
%load_ext memray

In [9]:
test_pv = pv.Image.new_from_file("../LR10_N554_S14_HE.jpg") / 255
test_pv = test_pv.resize(0.5, kernel="linear")
test = test_pv.numpy()
mask = get_tissue_mask(test, sigma=1)[-1]
print(test.shape, test.dtype, test.min(), test.max())
test = test.astype(np.float64) * 255
print(test.shape, test.dtype, test.min(), test.max())
# print(mask.shape, mask.dtype, mask.min(), mask.max())
# # mask=None

(12300, 11249, 3) float32 0.0006127451 1.0
(12300, 11249, 3) float64 0.1562500055297278 255.0


In [3]:
x = htk.preprocessing.color_conversion.rgb_to_sda(test, I_0=None, allow_negatives=False)
y = htk.preprocessing.color_conversion.rgb_to_sda_rs(test, I_0=None, allow_negatives=False)
precision64 = x != y
precision32 = x.astype(np.float32) != y.astype(np.float32)
print(precision64.sum())
print(precision32.sum())

153045220
0


In [4]:
x1 = htk.preprocessing.color_deconvolution.separate_stains_macenko_pca(x)
y1 = htk.preprocessing.color_deconvolution.separate_stains_macenko_pca_rs(y)
precision64 = x1 != y1
precision32 = x1.astype(np.float32) != y1.astype(np.float32)
print(x1[0,0])
print(y1[0,0])
print(precision64.sum())
print(precision32.sum())

0.14056042235863808
0.14056042234847294
9
0


In [10]:
%%memray_flamegraph --native
start = time()
rs_pca = htk.preprocessing.color_deconvolution.rgb_separate_stains_macenko_pca_rs(
    test, 
    255, 
    mask_out=mask
)
print("Rust PCA took", time() - start, "seconds")

/Users/mjbarrett/Code/rustification/pca_deconv/HistomicsTK/.venv/lib/python3.13/site-packages/rich/live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Rust PCA took 2.5106089115142822 seconds


⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately reflect stack traces. Please use an 
interpreter built with debug symbols for best results. Check https://bloomberg.github.io/memray/native_mode.html 
for more information regarding how memray resolves symbols.

Results saved to 
/Users/mjbarrett/Code/rustification/pca_deconv/HistomicsTK/memray-results/tmp9k7e38kj/flamegraph.html

In [11]:
%%memray_flamegraph --native
start = time()
htk_pca = htk.preprocessing.color_deconvolution.rgb_separate_stains_macenko_pca(
    test, 
    255, 
    mask_out=mask
)
print("HTK PCA took", time() - start, "seconds")

HTK PCA took 11.010789155960083 seconds


⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately reflect stack traces. Please use an 
interpreter built with debug symbols for best results. Check https://bloomberg.github.io/memray/native_mode.html 
for more information regarding how memray resolves symbols.

Results saved to 
/Users/mjbarrett/Code/rustification/pca_deconv/HistomicsTK/memray-results/tmpfz9ntpye/flamegraph.html

In [ ]:
precision64 = htk_pca != rs_pca
precision32 = htk_pca.astype(np.float32) != rs_pca.astype(np.float32)
print(htk_pca[0,0])
print(rs_pca[0,0])
print(precision64.sum())
print(precision32.sum())
print("htk: \n", htk_pca)
print("rust: \n", rs_pca)

0.25888478439628576
0.2588847843942145
9
0
rust: 
 [[ 0.25888478  0.56856653  0.67569952]
 [ 0.8973328   0.56304892  0.09620101]
 [ 0.3574528   0.59975663 -0.73087313]]
htk: 
 [[ 0.25888478  0.56856653  0.67569952]
 [ 0.8973328   0.56304892  0.09620101]
 [ 0.3574528   0.59975663 -0.73087313]]


In [ ]:
np.save("test_image.npy", test)

In [ ]:
# use matplotlib to visualize the two different PCA results, showing the the three channels of both methods independently
import matplotlib.pyplot as plt
htk_stains = htk.preprocessing.color_deconvolution.color_deconvolution(test, htk_pca).Stains
rs_stains = htk.preprocessing.color_deconvolution.color_deconvolution(test, rs_pca).Stains
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
axs[0, 0].imshow(htk_stains[:, :, 0], cmap='gray')
axs[0, 0].set_title('HTK Stain 1')
axs[0, 1].imshow(htk_stains[:, :, 1], cmap='gray')
axs[0, 1].set_title('HTK Stain 2')
axs[0, 2].imshow(htk_stains[:, :, 2], cmap='gray')
axs[0, 2].set_title('HTK Stain 3')

axs[1, 0].imshow(rs_stains[:, :, 0], cmap='gray')
axs[1, 0].set_title('Rust Stain 1')
axs[1, 1].imshow(rs_stains[:, :, 1], cmap='gray')
axs[1, 1].set_title('Rust Stain 2')
axs[1, 2].imshow(rs_stains[:, :, 2], cmap='gray')
axs[1, 2].set_title('Rust Stain 3')

In [ ]:
# visualize each channel of each deconv, but just for a tile at the center of the image:
tile_size = 1024
center_x, center_y = test.shape[1] // 2 + tile_size, test.shape[0] // 2 + tile_size
start_x = center_x - tile_size // 2
start_y = center_y - tile_size // 2

htk_tile = htk_stains[start_y:start_y + tile_size, start_x:start_x + tile_size, :]
rs_tile = rs_stains[start_y:start_y + tile_size, start_x:start_x + tile_size, :]
orig_image_tile = test[start_y:start_y + tile_size, start_x:start_x + tile_size, :]
mask_tile = mask[start_y:start_y + tile_size, start_x:start_x + tile_size]

fig, axs = plt.subplots(2, 4, figsize=(15, 10))
axs[0, 0].imshow(htk_tile[:, :, 0], cmap='gray')
axs[0, 0].set_title('HTK Tile Stain 1')
axs[0, 1].imshow(htk_tile[:, :, 1], cmap='gray')
axs[0, 1].set_title('HTK Tile Stain 2')
axs[0, 2].imshow(htk_tile[:, :, 2], cmap='gray')
axs[0, 2].set_title('HTK Tile Stain 3')
axs[0, 3].imshow(orig_image_tile)
axs[0, 3].set_title("Original Image")

axs[1, 0].imshow(rs_tile[:, :, 0], cmap='gray')
axs[1, 0].set_title('Rust Tile Stain 1')
axs[1, 1].imshow(rs_tile[:, :, 1], cmap='gray')
axs[1, 1].set_title('Rust Tile Stain 2')
axs[1, 2].imshow(rs_tile[:, :, 2], cmap='gray')
axs[1, 2].set_title('Rust Tile Stain 3')
axs[1, 3].imshow(mask_tile)
axs[1, 3].set_title("Mask")

plt.tight_layout()
plt.show()

In [ ]:
print(rs_pca[0].dtype, rs_pca[0].min(), rs_pca[0].max())
print(htk_pca[0].dtype, htk_pca[0].min(), htk_pca[0].max())
print(rs_pca[1].dtype, rs_pca[1].min(), rs_pca[1].max())
print(htk_pca[1].dtype, htk_pca[1].min(), htk_pca[1].max())
print(rs_pca[2].dtype, rs_pca[2].min(), rs_pca[2].max())
print(htk_pca[2].dtype, htk_pca[2].min(), htk_pca[2].max())